In [1]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.3 MB/s eta 0:00:00:00:0100:01


In [2]:
import os
import json
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
#images- 10k (500 provided by guild, rest generated)
images_dir = "/kaggle/input/ai-guild-pokemon-hackathon-training-images/Entire generated dataset/images"  
json_file  = "/kaggle/input/ai-guild-pokemon-hackathon-training-images/Entire generated dataset/coco_with_aug2.json"

with open(json_file, "r") as f:
    coco = json.load(f)

images = {img["id"]: img["file_name"] for img in coco["images"]}
annotations = coco["annotations"]

#class mapping (categories)
categories = {cat["id"]: cat["name"] for cat in coco["categories"]}
print("Categories:", categories)


Categories: {1: 'Pikachu', 2: 'Charizard', 3: 'Bulbasaur', 4: 'Mewtwo'}


In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split


yolo_dataset = "/kaggle/working/yolo_pokemon"   #output for yolo
os.makedirs(f"{yolo_dataset}/images/train", exist_ok=True)
os.makedirs(f"{yolo_dataset}/images/val", exist_ok=True)
os.makedirs(f"{yolo_dataset}/labels/train", exist_ok=True)
os.makedirs(f"{yolo_dataset}/labels/val", exist_ok=True)

ann_by_image = {}
for ann in annotations:
    ann_by_image.setdefault(ann["image_id"], []).append(ann)

# Split train/val
image_ids = list(images.keys())
train_ids, val_ids = train_test_split(image_ids, test_size=0.2, random_state=42)

def coco_to_yolo(bbox, img_w, img_h):
    x, y, w, h = bbox
    xc = (x + w/2) / img_w
    yc = (y + h/2) / img_h
    return xc, yc, w/img_w, h/img_h

def save_labels(ids, split):
    for img_id in ids:
        img_file = images[img_id]
        img_path = os.path.join(images_dir, img_file)
        out_img_path = f"{yolo_dataset}/images/{split}/{img_file}"
        out_label_path = f"{yolo_dataset}/labels/{split}/{img_file.replace('.png', '.txt')}"

        # copy image safely
        shutil.copy(img_path, out_img_path)

        # write labels
        img_w, img_h = 640, 480  # adjust if metadata differs
        with open(out_label_path, "w") as f:
            for ann in ann_by_image.get(img_id, []):
                cls = ann["category_id"] - 1  # YOLO classes start at 0
                x, y, w, h = coco_to_yolo(ann["bbox"], img_w, img_h)
                f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

save_labels(train_ids, "train")
save_labels(val_ids, "val")


In [6]:
data_yaml = f"""
train: {yolo_dataset}/images/train
val: {yolo_dataset}/images/val
nc: {len(categories)}
names: {list(categories.values())}
"""

with open(f"{yolo_dataset}/data.yaml", "w") as f:
    f.write(data_yaml)

print(open(f"{yolo_dataset}/data.yaml").read())



train: /kaggle/working/yolo_pokemon/images/train
val: /kaggle/working/yolo_pokemon/images/val
nc: 4
names: ['Pikachu', 'Charizard', 'Bulbasaur', 'Mewtwo']



In [7]:
model = YOLO("yolo12x.pt")
model.train(
    data=f"{yolo_dataset}/data.yaml",
    epochs=5,
    imgsz=640,
    batch=5
)

Ultralytics 8.3.202 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=5, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_pokemon/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       3179      13127      0.997      0.995      0.995      0.964
               Pikachu       1955       3563      0.994      0.994      0.995      0.957
             Charizard       1866       3368          1      0.994      0.995      0.973
             Bulbasaur       1838       3266      0.998      0.996      0.995      0.958
                Mewtwo       1727       2930      0.998      0.997      0.995      0.967
Speed: 0.2ms preprocess, 32.5ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /kaggle/working/runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79dc2a4ecc50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [8]:
"""
    YOLO inference script
"""
from ultralytics import YOLO
import os, glob
import pandas as pd

# Loading best weights
model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")

img_folder = "/kaggle/input/final-test/test_images"
img_paths = glob.glob(os.path.join(img_folder, "*.png")) + glob.glob(os.path.join(img_folder, "*.jpg"))

rows = []

for img_path in img_paths:
    results = model.predict(img_path, conf=0.5, save=False, verbose=False)
    res = results[0]

    image_id = os.path.basename(img_path)

    # Iterate over detections
    for box in res.boxes:
        cls_id = int(box.cls.item())  
        x_center, y_center, width, height = box.xywh[0].tolist()

        rows.append({
            "image_id": image_id,
            "pokemon_id": cls_id,
            "x_center": x_center,
            "y_center": y_center,
            "width": width,
            "height": height
        })

# Save to CSV
df = pd.DataFrame(rows)
csv_path = "/kaggle/working/predictions.csv"
df.to_csv(csv_path, index=False)

print(f"✅ Saved predictions to {csv_path}")
df.head()


✅ Saved predictions to /kaggle/working/predictions.csv


,image_id,pokemon_id,x_center,y_center,width,height
0,img_00100.png,2,487.357758,237.769958,156.215637,147.855652
1,img_00100.png,1,549.335205,57.001141,94.086151,92.847115
2,img_00100.png,2,309.240906,115.446625,115.607559,145.094025
3,img_00100.png,2,129.298615,375.764191,124.204155,142.595520
4,img_00100.png,1,142.773926,161.150620,59.445747,60.337189


In [ ]:
from ultralytics import YOLO
import os, glob
from IPython.display import Image, display

def test_images(img_paths, conf=0.5):
    results = model.predict(img_paths, conf=conf, save=True)
    
    for res in results:
        save_dir = str(res.save_dir)
        # Try exact match
        original_name = os.path.basename(res.path)
        out_path = os.path.join(save_dir, original_name)

        # If file not found, search by pattern (YOLO sometimes prefixes folders)
        if not os.path.exists(out_path):
            pattern = os.path.splitext(original_name)[0]  # base name without extension
            candidates = glob.glob(f"{save_dir}/*{pattern}*")
            if candidates:
                out_path = candidates[0]  # first match

        print(f"✅ Predictions saved at: {out_path}")
        display(Image(filename=out_path))

# ---- Test multiple images ----
test_images([
    "/kaggle/input/final-test/test_images/img_00000.png",
    "/kaggle/input/final-test/test_images/img_00001.png",
    "/kaggle/input/final-test/test_images/img_00007.png", 
    "/kaggle/input/final-test/test_images/img_00020.png",
    "/kaggle/input/final-test/test_images/img_00029.png",
    "/kaggle/input/final-test/test_images/img_00018.png"
])
